In [ ]:
import os
import sys
import json
import pathlib

from math import sqrt
import joblib
from joblib import Parallel, delayed
import time

from wasabi import msg

from functools import lru_cache

import pandas as pd
import numpy as np

from typing import Union, List, Dict, Optional


In [ ]:
COUNT = 500000

In [ ]:
s = time.time()
ns = [sqrt(i**2) for i in range(COUNT)]
print(time.time() - s)

In [ ]:
len(ns), ns[0]

In [ ]:
import concurrent.futures

In [ ]:
with joblib.parallel_backend(backend='multiprocessing', n_jobs=3):
    s = time.time()
    ns = Parallel(n_jobs=3)(delayed(sqrt)(i**2) for i in range(COUNT))
    print(time.time() - s)

In [ ]:
len(ns), ns[0]

In [ ]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
from pathlib import Path
import os
from datetime import datetime

from wasabi import msg

import fgscraper.common.utils as spyder_utils
from fgscraper.common.data_ingest_manager import DataIngestManager

ROOT_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), '../')
DATA_PATH = Path(ROOT_PATH)/'data'
data_manager = DataIngestManager(
    raw_enterprise_path=DATA_PATH/'raw_enterprise')
file_paths = data_manager.get_file_paths(DATA_PATH/'enterprise_ids', 'txt')
dataset = spyder_utils.gen_dataset(file_paths)

async def run(playwright):
    if not len(file_paths):
        msg.fail('No enterprise ids provided. Be sure to have run fg_post_spider.py before.')
        
    for data in dataset:
        chromium = playwright.chromium
        browser = await chromium.launch()
        fpath = Path(data[0])
        regione, provincia = fpath.stem.split('_')[1:]
        id_lines = data[1].splitlines()
        start_n_lines = len(id_lines)
        msg.info(f'===== Processing files with {start_n_lines} Ids')
        msg.info(f'regione: {regione}, provincia: {provincia}')
        # check if raw enterprise json already exist
        already_ids = data_manager.get_raw_enterprise_paths()
        msg.good(f'Checking if current TXT contains already processed enterprise ...')
        id_lines = list(set(id_lines).difference(already_ids))  # ids are unique, so sets could be used. 
        msg.good(f'id_lines reduced of {start_n_lines - len(id_lines)}')

        for id_line in id_lines:
            _url = spyder_utils.get_full_url(id_line)
            msg.info(f'requesting page at {_url}')
            try:
                context = await browser.new_context()  # create a new incognite mode context
                page = await context.new_page()
                await page.goto(_url)
                html = await page.inner_html('#main_content')
                soup = BeautifulSoup(html, 'html.parser')
            except PlaywrightTimeoutError as e:
                msg.fail(e)
                continue

            # get cols and values
            data1 = spyder_utils.extract_table1_values(_soup=soup)
            data2 = spyder_utils.extract_table2_values(_soup=soup)
            if not data1 and not data2:
                msg.warn(
                    f'Problems with {_url}: neither table1 nor table2 founded.')
                continue

            full_dict = spyder_utils.get_full_dict(data1, data2)
            # Make two dictionaries and join them
            full_dict.update({'Regione': regione.upper(),
                             'Provincia': provincia.upper(),
                              'source_file': fpath.name,
                              'EnterpriseID': id_line})
            msg.info(f'Retrived full dict: {full_dict}')
            # write the raw enterprise info to the disk
            now = datetime.strftime(datetime.now(), '%Y%m%d%H%M')
            fname_raw = f'{now}_{regione}_{provincia}_{id_line}'
            try:
                await data_manager.write_json(
                    full_dict=full_dict, dir_path=DATA_PATH/'raw_enterprise', fname=fname_raw)
            except PermissionError as e:
                msg.fail(e)
            await context.close()
            
        await browser.close()


async def main():
    async with async_playwright() as playwright:
        await run(playwright)


In [ ]:
def get_file_paths(data_path: Union[str, pathlib.Path], file_prefix: str, sort: Optional[bool] = True) -> List[str]:
    """
    Util for getting a sorted/unsorted list of file paths from disk.

        --Parameters
     - data_path: (str, or Path), the directory path
     - file_prefix: (str), the file prefixs which should be searched (txt, json, csv, etc ..)
     - sort: (bool), if or not returning a sorted lis

        --Return
     - A list containing the founded file with prefix file_prefix

        """

    if isinstance(data_path, str):
        data_path = pathlib.Path(data_path)

    if not data_path.is_dir():
        raise ValueError(f'data path {data_path} is not a regular directory.')
    
    file_prefix = file_prefix if not file_prefix[0] == '.' else file_prefix[1:]
    ls = list(map(lambda x: x, data_path.glob(f'*.{file_prefix}')))
    msg.info(f'Founded {len(ls)} file with prefix {file_prefix} at {data_path}')
    if sort:
        return sorted(ls)
    return ls

@lru_cache
def get_raw_enterprise_paths():
    enterprise_ids_paths = get_file_paths(data_path='../../data/raw_enterprise', file_prefix='json')
    ent_ids = list(map(lambda x: str(x.stem).split('_')[3], enterprise_ids_paths)) # sorted enterprise ids
    msg.info(f'ENT ID: {ent_ids[0]}')
    return sorted(ent_ids)

def check_raw_enterprise_exist(enterprise_id: str):
    ent_ids = get_raw_enterprise_paths()
    msg.info(f'Looking if {enterprise_id} has been already processed')
    return binary_search(array=ent_ids, item=enterprise_id)

def binary_search(array, item):
    left = 0
    right = len(array) - 1
    if item < array[left] or item > array[right]:
        return None
    while left <= right:
        mid = left + (right - left) // 2
        el = array[mid]
        print(el)
        print(mid)
        print(right)
        print(left)
        if el == item:
            msg.good('found')
            return True
        elif el > item:
            right = mid - 1
        else:
            left = mid + 1
    return None

In [ ]:
with open('../../data/enterprise_ids/202207211828_Abruzzo_AQUILA.txt') as f:
    txt = f.read().splitlines()
txt[0], txt[1]

In [ ]:
ent_ids = get_raw_enterprise_paths()

In [ ]:
list(set(txt).difference(ent_ids))

In [ ]:
ent_ids[8] < ent_ids[52]

In [ ]:
df = pd.read_csv('../../data/final_df_prova.csv', index_col=0)
df.head()

In [ ]:
# ROOT_PATH = '../..'
# DATA_PATH = Path(ROOT_PATH)/'data'
# file_paths = sorted(list(map(lambda x: str(x), Path(DATA_PATH).glob('*.txt'))))
# logger.info(f'Retrived {len(file_paths)} txt files in {DATA_PATH}')

# BASE_URL = 'https://www.fgas.it/RicercaSezC/DettaglioImpresa?id='


# def gen_dataset(file_paths: str):
#     for f_path in file_paths:
#         with open(f_path) as f:
#             logger.info(f'yielding {f_path}')
#             yield (f_path, f.read())


# def get_full_url(enterprise_id: str) -> str:
#     return BASE_URL + enterprise_id

In [ ]:
# dataset = gen_dataset(file_paths)

# for data in dataset:
#     p = Path(data[0])
#     print(p.parent)
#     ddd = p.stem.split('_')
#     print(ddd[0], ddd[1], ddd[2])
#     print(p.name)
#     lines = data[1].splitlines()
#     print(len(lines))
#     for line in lines:
#         print(line)
#         break
#     break

In [ ]:
payload_dict = {
    'FormatoReport': '3',
    'GeneraReport': 'false',
    'displayReport': 'display%3Anone%3B',
    'flagAreaPub': 'True',
    'DataFromSession': 'false',
    'DownloadToken': '',
    'NumRecord': '160',
    'Nazionalita': 'I',
    'IDRegione': '19',
    'IstatProv': '083',
    'Identificativo': '',
    'RadioBtnDenominazione': 'C',
    'Denominazione': '',
    'NumCertProv': '',
    'TipoSoggetto': 'I',
    'IDAttivita_I': '',
    'IDAttivita': ''}

raw_payload = "FormatoReport=3&GeneraReport=false&displayReport=display%3Anone%3B&flagAreaPub=True&DataFromSession=false&DownloadToken=&NumRecord=160&Nazionalita=I&IDRegione=19&IstatProv=083&Identificativo=&RadioBtnDenominazione=C&Denominazione=&NumCertProv=&TipoSoggetto=I&IDAttivita_I=&IDAttivita="


In [ ]:
s = [key + '=' + value for key, value in payload_dict.items()]
ss = '&'.join(s)
print(ss)
print(raw_payload)
assert ss == raw_payload

In [ ]:
with open('../../assets/regions-ids.json') as f:
    regions = json.load(f)

with open('../../assets/provinces-ids.json') as f:
    provinces = json.load(f)
len(regions), len(provinces)

In [ ]:
data_path = '../../data/202207181700_Abruzzo_AQUILA.json'
with open(data_path) as f:
    d = json.loads(json.load(f))